# Synthesis Planning
This notebook goes through the whole process to use the optimized model and produce the synthesis planning.

## Environment Setup
First of all, we need to make sure that the notebook is running in the correct environment.

To do that, follow these steps:
 1. Create the project's environment
    To do that, place yourself in the project's root, and run :
    `conda env create -f environment.yml`
    This creates a new clean conda environment with the package needed by the project.
 2. Activate the environment
    On Linux and Mac:
    `source activate synnet`
    On Windows:
    `conda activate synnet`
 3. Install the project's module
    Now that the environment is activated, we need to install the project as a module.
    Place yourself in the project's root and run :
    `pip install -e .`
 4. Restart Jupyter from the new environment
    Now, we can start Jupyter from the environment, that way it has all the dependencies we need. Simply run `jupyter notebook` and open this notebook.

To test the setup, run the following cell.

In [1]:
import sys

from helpers import paths

# Check that the correct conda env is being used
if sys.prefix.split("\\")[-1] != "synnet":
    print(
        "You are not using the correct conda environment, please follow the instructions above"
    )
else:
    try:
        import synnet

        print("The environment is setup correctly")
    except ImportError:
        print(
            "The module 'synnet' is not installed, please follow the instructions above"
        )

You are not using the correct conda environment, please follow the instructions above


## Pre-Processing

Now that the conda environment is correctly setup, we can start the preliminary steps to produce the synthesis results.

First, let's import some packages, define some constants.
Make sure they are correct.

In [2]:
from helpers.loader import *
from helpers.preprocessor import *
from helpers.synthesis import synthesis
from helpers.optimize import optimize

# Number of cores to use for the computation. The greater, the faster
cpu_cores = 20
# Number of molecules to randomly pick from the datasets.
# Our results were made with a sample of 10000
num_samples = 10000
# Seed to use to sample the datasets
seed = 42

### Load data
First, we need to choose the trained model to use.

For now, we only have the one provided by the paper's authors

In [10]:
original_checkpoints = get_original_checkpoints()

original_checkpoints.tar.gz is already present, no need to download it


Extracting original_checkpoints.tar.gz: 100%|██████████| 1.70G/1.70G [00:08<00:00, 211MB/s]


And the model we trained

In [4]:
trained_checkpoints = get_trained_checkpoints()

act.tar.gz is already present, no need to download it


Extracting act.tar.gz: 100%|██████████| 164M/164M [00:00<00:00, 221MB/s] 


rt1.tar.gz is already present, no need to download it


Extracting rt1.tar.gz: 100%|██████████| 172M/172M [00:00<00:00, 193MB/s] 


rxn.tar.gz is already present, no need to download it


Extracting rxn.tar.gz: 100%|██████████| 815M/815M [00:03<00:00, 229MB/s] 


rt2.tar.gz is already present, no need to download it


Extracting rt2.tar.gz: 100%|██████████| 804M/804M [00:03<00:00, 222MB/s] 


Now, we need to retrieve the building blocks. We asked the company to provide them, that way we can correctly reproduce their result

To simplify the workflow, this also perform the step 0 described in INSTRUCTIONS.md

In [5]:
bblocks_raw = get_building_blocks()

Loading sdf...  done


Converting sdf to smile: 100%|██████████| 195944/195944 [00:17<00:00, 11034.37mol/s]


We also need to download the molecules we want to test the model on.

We will use three datasets :
 - the reachable molecules
 - the ZINC dataset
  - the ChEMBL dataset

In [6]:
# Our reachable set is generated using a sample size of 10000 and a seed of 42
test_smiles = get_reachable_dataset(num_samples, seed)
zinc_smiles = get_zinc_dataset(num_samples, seed)
chembl_smiles = get_chembl_dataset(num_samples, seed)

Loading reachable set...


chembl_31.sdf.gz is already present, no need to download it
chembl_31.sdf is already present, no need to extract it
Loading sdf...  done


Converting sdf to smile: 100%|██████████| 10000/10000 [00:01<00:00, 5440.25mol/s]


## Process Building Blocks

### Filter Building Blocks
First, we apply the step 1 from INSTRUCTION.md

We pre-process the building blocks to identify applicable reactants for each reaction template. In other words, filter out all building blocks that do not match any reaction template. There is no need to keep them, as they cannot act as reactant.

In a first step, we match all building blocks with each reaction template.
In a second step, we save all matched building blocks and a collection of `Reactions` with their available building blocks.

In [7]:
bblocks, rxn_collection = filter_bblocks(bblocks_raw)

[16:58:43] mapped atoms in the reactants were not mapped in the products.
  unmapped numbers are: 11 
[16:58:43] mapped atoms in the reactants were not mapped in the products.
  unmapped numbers are: 11 
[16:58:43] mapped atoms in the reactants were not mapped in the products.
  unmapped numbers are: 11 
[16:58:43] mapped atoms in the reactants were not mapped in the products.
  unmapped numbers are: 18 16 
[16:58:43] mapped atoms in the reactants were not mapped in the products.
  unmapped numbers are: 1 14 
[16:58:43] product atom-mapping number 13 not found in reactants.
[16:58:43] mapped atoms in the reactants were not mapped in the products.
  unmapped numbers are: 11 
[16:58:43] product atom-mapping number 10 not found in reactants.
[16:58:43] product atom-mapping number 11 not found in reactants.
[16:58:43] product atom-mapping number 12 not found in reactants.
[16:58:43] product atom-mapping number 13 not found in reactants.
[16:58:43] product atom-mapping number 14 not found i

### Pre-compute embeddings
Then, step 2

We use the embedding space for the building blocks a lot. Hence, we pre-compute and store the building blocks.

In [8]:
mol_embedder = compute_embeddings(bblocks, cpu_cores)

17:05:51 synnet.MolEmbedder INFO: Will compute embedding with 20 processes.
17:05:57 synnet.MolEmbedder INFO: Computed embeddings.
17:05:57 synnet.MolEmbedder INFO: Successfully saved data (shape=(178035, 256)) to data/pre-process/embeddings/hb-enamine-embeddings.npy .


# Synthesis

Now that everything is loaded and pre-processed, we can do the synthesis prediction.

First, we compute synthetic trees for the reachable smiles, both on the original and the trained checkpoints

In [ ]:
synthesis(
    test_smiles,
    bblocks,
    original_checkpoints,
    rxn_collection,
    mol_embedder,
    paths.synthesis_result_path("reachable", "original"),
    rxn_template="hb",
    n_bits=4096,
    beam_width=3,
    max_step=15,
    cpu_cores=cpu_cores,
)

In [ ]:
synthesis(
    test_smiles,
    bblocks,
    trained_checkpoints,
    rxn_collection,
    mol_embedder,
    paths.synthesis_result_path("reachable", "trained"),
    rxn_template="hb",
    n_bits=4096,
    beam_width=3,
    max_step=15,
    cpu_cores=cpu_cores,
)

Compute synthetic trees for the ZINC dataset, both on the original and the trained checkpoints

In [ ]:
synthesis(
    zinc_smiles,
    bblocks,
    original_checkpoints,
    rxn_collection,
    mol_embedder,
    paths.synthesis_result_path("zinc", "original"),
    rxn_template="hb",
    n_bits=4096,
    beam_width=3,
    max_step=15,
    cpu_cores=cpu_cores,
)

In [ ]:
synthesis(
    zinc_smiles,
    bblocks,
    trained_checkpoints,
    rxn_collection,
    mol_embedder,
    paths.synthesis_result_path("zinc", "trained"),
    rxn_template="hb",
    n_bits=4096,
    beam_width=3,
    max_step=15,
    cpu_cores=cpu_cores,
)

Compute synthetic trees for the ChEMBL dataset

In [ ]:
synthesis(
    chembl_smiles,
    bblocks,
    original_checkpoints,
    rxn_collection,
    mol_embedder,
    paths.synthesis_result_path("chembl", "original"),
    rxn_template="hb",
    n_bits=4096,
    beam_width=3,
    max_step=15,
    cpu_cores=cpu_cores,
)

In [ ]:
synthesis(
    chembl_smiles,
    bblocks,
    trained_checkpoints,
    rxn_collection,
    mol_embedder,
    paths.synthesis_result_path("chembl", "trained"),
    rxn_template="hb",
    n_bits=4096,
    beam_width=3,
    max_step=15,
    cpu_cores=cpu_cores,
)

## Optimize

In [15]:
optimize(
    zinc_smiles[:5],
    bblocks,
    rxn_collection,
    original_checkpoints,
    mol_embedder,
    paths.optimize_result_path('zinc', 'original'),
    nbits=4096,
    num_gen=60,
    objective="gsk",
    rxn_template="hb",
    num_offspring=128,
    cpu_cores=cpu_cores,
)

/tmp/ipykernel_908113/3537202672.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  zinc_smiles[:5],


Start.
Data loaded


Found local copy...


Initial: 0.022 +/- 0.024
Scores: [0.06 0.04 0.01 0.   0.  ]
Top-3 Smiles: ['O=C(c1ccnnc1)N1CCOCC1c1ccn[nH]1', 'O=C(COc1ccc(F)cc1)NC1CCCCNC1=O', 'CC(C)CS(=O)(=O)Nc1ccc(C(=O)O)cc1']
Starting generation 0


Found local copy...


...

Generation 35: 0.756 +/- 0.016
Scores: [0.78 0.77 0.75 0.74 0.74]
Top-3 Smiles: ['CC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(N3CCn4c(nc(C(F)F)c4C(=O)O)C3)ccc21', 'CNC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(C3=C(CC(=O)O)CCC3)ccc21', 'CNC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(C3=C(CC(=O)O)CCOC3)ccc21']
Consumed time: 68.491 s

===============Stored Molecules===============
O=C(O)c1ccc2[nH]ncc2c1 False
CC(=O)Nc1ccc(Cl)cc1N False
CC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(Cl)ccc21 False
Cl.Cl.O=C(O)c1c(C(F)F)nc2n1CCNC2 False
CC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(N3CCn4c(nc(C(F)F)c4C(=O)O)C3)ccc21 False
CCOC(=O)C1CC(Br)=NO1 False
O=C(O)c1c(C(F)F)nc2n1CCN(c1ccc3c(c1)nc(-c1ccc4[nH]ncc4c1)n3C(=O)CC(=O)C1CC(Br)=NO1)C2 True
===============Stored Reactions===============
1 2
41 2
69 2
===============Followed Actions===============
[0, 1, 1, 3]
===============Stored Molecules===============
O=C(O)c1ccc2[nH]ncc2c1 False
CNC(=O)Nc1ccc(Br)cc1N False
CNC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(Br)ccc21 False
O=C(O)CC1=CCCC1 False
CNC(=O)n1c(-c2cc

Found local copy...


Generation 36: 0.756 +/- 0.016
Scores: [0.78 0.77 0.75 0.74 0.74]
Top-3 Smiles: ['CC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(N3CCn4c(nc(C(F)F)c4C(=O)O)C3)ccc21', 'CNC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(C3=C(CC(=O)O)CCC3)ccc21', 'CNC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(C3=C(CC(=O)O)CCOC3)ccc21']
Consumed time: 66.476 s

===============Stored Molecules===============
O=C(O)c1ccc2[nH]ncc2c1 False
CC(=O)Nc1ccc(Cl)cc1N False
CC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(Cl)ccc21 False
Cl.Cl.O=C(O)c1c(C(F)F)nc2n1CCNC2 False
CC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(N3CCn4c(nc(C(F)F)c4C(=O)O)C3)ccc21 False
CCOC(=O)C1CC(Br)=NO1 False
O=C(O)c1c(C(F)F)nc2n1CCN(c1ccc3c(c1)nc(-c1ccc4[nH]ncc4c1)n3C(=O)CC(=O)C1CC(Br)=NO1)C2 True
===============Stored Reactions===============
1 2
41 2
69 2
===============Followed Actions===============
[0, 1, 1, 3]
===============Stored Molecules===============
O=C(O)c1ccc2[nH]ncc2c1 False
CNC(=O)Nc1ccc(Br)cc1N False
CNC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(Br)ccc21 False
O=C(O)CC1=CCCC1 False
CNC(=O)n1c(-c2cc

Found local copy...


Generation 37: 0.756 +/- 0.016
Scores: [0.78 0.77 0.75 0.74 0.74]
Top-3 Smiles: ['CC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(N3CCn4c(nc(C(F)F)c4C(=O)O)C3)ccc21', 'CNC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(C3=C(CC(=O)O)CCC3)ccc21', 'CNC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(C3=C(CC(=O)O)CCOC3)ccc21']
Consumed time: 65.603 s

===============Stored Molecules===============
O=C(O)c1ccc2[nH]ncc2c1 False
CC(=O)Nc1ccc(Cl)cc1N False
CC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(Cl)ccc21 False
Cl.Cl.O=C(O)c1c(C(F)F)nc2n1CCNC2 False
CC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(N3CCn4c(nc(C(F)F)c4C(=O)O)C3)ccc21 False
CCOC(=O)C1CC(Br)=NO1 False
O=C(O)c1c(C(F)F)nc2n1CCN(c1ccc3c(c1)nc(-c1ccc4[nH]ncc4c1)n3C(=O)CC(=O)C1CC(Br)=NO1)C2 True
===============Stored Reactions===============
1 2
41 2
69 2
===============Followed Actions===============
[0, 1, 1, 3]
===============Stored Molecules===============
O=C(O)c1ccc2[nH]ncc2c1 False
CNC(=O)Nc1ccc(Br)cc1N False
CNC(=O)n1c(-c2ccc3[nH]ncc3c2)nc2cc(Br)ccc21 False
O=C(O)CC1=CCCC1 False
CNC(=O)n1c(-c2cc